In [ ]:
# Will want to execute this in docker so we don't have to include BeautifulSoup in Spark.
from bs4 import BeautifulSoup

with open("download_html/os_download_links_page.htm") as f:
    soup = BeautifulSoup(f.read())

soup.find("a")

el = soup.find(text = "Number of Files:").parent.parent
num = el.text.replace("Number of Files:", "").strip()
numfiles = int(num)

my_links = set()
for a in soup.findAll("a"):
    if "href" in a.attrs:
        if "AB76DL" in a["href"]:  # This is a bit of trial and error, but it turned out that the download links all contain "AB76DL" as part of the URL
            my_links.add(a["href"])
my_links = list(my_links)

# Check we've found the right number of links
if (len(my_links) != numfiles):
    raise Exception

In [ ]:
from pyspark.sql import functions as fns
from pyspark import  SparkContext
from pyspark.sql import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

# For py 2 and 3 compataibility
try:
    import urllib.request as urlrequest
except ImportError:
    import urllib as urlrequest
    
import re




def download_and_put_in_s3(link):
    
    #Must be within function otherwise tries to pickle and send boto3 client to workers
    import boto3
    s3_client = boto3.client('s3')
    
    # To do - make this work in Python 2 so that it runs on glue
    match = re.search("CSV\/(\w{2}\d{4}\.zip)", link)[1]
    urlrequest.urlretrieve(link, "{}".format(match))
    
    s3_client.upload_file(match, "alpha-everyone", "deleteathenaout/abpzips/{}".format(match))
    
    # Put in s3
    return match


rdd = sc.parallelize(my_links[:1000], numSlices=100)  # As I understand it, if we set num slices >= num cpus in the cluster, we will parallelise across all cpus
downloaded_files = rdd.map(download_and_put_in_s3).collect()